In [106]:
import requests
import pandas as pd
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

# df['salaire_minimum'] = df['lieu'].apply(lambda x: x[1] if len(x) > 1 else '')
# df["salaire_maximum"] = df["salaire_minimum"].apply(lambda x: x.split("-")[1].strip() if '-' in x else '')
# df["salaire_minimum"] = df["salaire_minimum"].apply(lambda x: x.split("-")[0].strip() if '-' in x else x)

df["salaire_minimum"], df["salaire_maximum"] = df["lieu"].apply(lambda x: (x[1].split("-")[0].strip(), x[1].split("-")[1].strip()) if len(x) > 1 and '-' in x[1] else ('', '') ).str
df["salaire_minimum"] = df["salaire_minimum"].apply(lambda x: x.replace(',','.').replace('€','').replace(' ','').replace('/an','') if x and '-' not in x else x)
df["salaire_maximum"] = df["salaire_maximum"].apply(lambda x: x.replace(',','.').replace('€','').replace(' ','').replace('/an','') if x and '-' not in x else x)
df["salaire_maximum"] = df["salaire_maximum"].replace('selonprofil', '', regex=True)


df["salaire_minimum"] = df["salaire_minimum"].replace('.000', '000', regex=True)
df["salaire_maximum"] = df["salaire_maximum"].replace('.000', '000', regex=True)
# there is one line in my dataframe where value is 30000  become 0 but i need it to stay 30000 the previous code is good for every other value but not for this one


# df["salaire_minimum"] = df["salaire_minimum"].replace('.00', '', regex=True)
# df["salaire_maximum"] = df["salaire_maximum"].replace('.00', '', regex=True)


# df["salaire_minimum"], df["salaire_maximum"] = df["lieu"].apply(lambda x: (x[1].split("-")[0].strip(), x[1].split("-")[1].strip()) if len(x) > 1 and '-' in x[1] else ('', '') ).str
# df["salaire_minimum"] = df["salaire_minimum"].apply(lambda x: x.replace(',','.').replace('€','').replace(' ','').replace('/an','').replace('.','') if x and '-' not in x else x)
# df["salaire_maximum"] = df["salaire_maximum"].apply(lambda x: x.replace(',','.').replace('€','').replace(' ','').replace('/an','').replace('.','') if x and '-' not in x else x)



df['lieu'] = df['lieu'].apply(lambda x : x[0])

df = df.apply(lambda c : c.apply(lambda x : ','.join([s.strip("\n ").lower() for s in (x.split(',') if type(x) != list else x)])))

df["Intitulé du poste"] = df["Intitulé du poste"].apply(lambda x: x.strip("(h/f)"))

df.to_csv("data_test1.csv", index=False)


C:\Users\loren\AppData\Local\Temp\ipykernel_15792\1781466382.py:14: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df["salaire_minimum"], df["salaire_maximum"] = df["lieu"].apply(lambda x: (x[1].split("-")[0].strip(), x[1].split("-")[1].strip()) if len(x) > 1 and '-' in x[1] else ('', '') ).str


In [100]:
df.dtypes

Intitulé du poste      object
Date de publication    object
lieu                   object
competences            object
Type de poste          object
salaire_minimum        object
salaire_maximum        object
dtype: object